In [1]:
from __future__ import print_function, division
import json 
import numpy as np
import cv2
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder
import time
import os
import copy
import sys

import shutil

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

import urllib.request as urllib2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
DATASET_PATH = './Dataset/'
DATASET_DIR = 'all/'

RESULT_PATH = './Result/'
RESULT_DIR = 'incorrect_img/'

In [3]:
classes = ['aeroplane','bicycle','diningtable',
           'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor',
           'bird','boat','bottle','bus','car','cat','chair','cow']

In [4]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [5]:
class SAVE_IMAGE:
    
    def __init__(self, ncols = 0, nrows = 0, figTitle="", axis = True):
        
        if ncols == 0 or nrows == 0:
            raise ValueError("ncols and nrows must be initialize")
        
        self.fig, self.ax = plt.subplots(ncols = ncols, nrows = nrows)
        self.ncols = ncols
        self.nrows = nrows
        
        if axis is False:
            plt.axis(False)
        
        if figTitle is not "":
            self.fig.suptitle(figTitle, fontsize=20)
        self.ccols = 0
        self.crows = 0
        
    def addImage(self, img, title = ""):
        
        if self.nrows == 1:
            if self.ncols == 1:
                self.ax.imshow(img)
                if title is not "":
                    self.ax.set_title(title, fontsize=15)
            else:
                self.ax[self.ccols].imshow(img)
                if title is not "":
                    self.ax[self.ccols].set_title(title, fontsize=15)
        else:
            self.ax[self.crows][self.ccols].imshow(img)
            if title is not "":
                self.ax[self.crows][self.ccols].set_title(title, fontsize=15)

        if self.ccols+1 == self.ncols:
            self.crows = self.crows + 1
            self.ccols = 0
        else:
            self.ccols = self.ccols + 1
            
    def showImage(self):
        plt.show()
        
    def saveImage(self, save_path, save_title):
        plt.savefig(save_path+save_title+'.png', bbox_inches='tight')
        plt.clf()

## 데이터 불러오기

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [7]:
image_datasets = {x: datasets.ImageFolder(os.path.join(DATASET_PATH+DATASET_DIR, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 모델구조 생성

### VGG16

In [8]:
"""
model = models.vgg16(pretrained=False)
model.classifier[-1] = nn.Linear(in_features=4096, out_features=len(class_names))
model = model.to(device)
"""

'\nmodel = models.vgg16(pretrained=False)\nmodel.classifier[-1] = nn.Linear(in_features=4096, out_features=len(class_names))\nmodel = model.to(device)\n'

### Resnet50

In [9]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 20)

model = model.to(device)
model.load_state_dict(torch.load('./TrainedModel/Resnet50_all'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## 모델 불러오기

In [10]:
def getImg(inp, title=None):
    """Imwrite for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    return inp

In [11]:
# save_parent_dir = answer
# save_child_dir = pred

def save_img(img, save_path, save_parent_dir, save_child_dir):
    
    plt.ioff()
        
    save_path = os.path.join(save_path, save_parent_dir, save_child_dir)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    img = getImg(img)
    save_title = str(len(os.listdir(save_path)))
    
    
    save_image = SAVE_IMAGE(nrows = 1, ncols = 1, figTitle=f"Ans: {classes[int(save_parent_dir)]}, Pred: {classes[int(save_child_dir)]}")
    #save_image = SAVE_IMAGE(nrows = 1, ncols = 1, axis = False)
    save_image.addImage(img)
    save_image.saveImage(save_path+'/', save_title)

In [12]:
def filter_out_incorrect_preds(inputs, preds, answer):
    for i in range(inputs.size()[0]):
        if preds[i] != answer.data[i]:
            save_img(inputs.cpu().data[i], DATASET_PATH+RESULT_PATH+RESULT_DIR,str(answer.data[i].item()), str(preds[i].item()))

In [13]:
def test_model(model):
    since = time.time()
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for index, (inputs, labels) in enumerate(dataloaders['test']):
            printProgressBar (index, len(dataloaders['test']), prefix='Progress', suffix='Complete', length=50)
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            
            total += labels.size(0)
            correct += torch.sum(preds == labels.data)
            
            filter_out_incorrect_preds(inputs, preds, labels)
            
    printProgressBar (len(dataloaders['test']), len(dataloaders['test']), prefix='Progress', suffix='Complete', length=50)
    time_elapsed = time.time() - since
    print('Test complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Test Acc: {:4f}'.format(correct.double() / dataset_sizes['test']))

In [14]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    flag = True
    
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                
                if flag:
                    print(inputs.cpu().data[j])
                    flag = False
                
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

## 모델 평가하기

In [15]:
#visualize_model(model)

In [16]:
if os.path.exists(DATASET_PATH+RESULT_PATH+RESULT_DIR):
        shutil.rmtree(DATASET_PATH+RESULT_PATH+RESULT_DIR)

test_model(model)

/home/lab405b/anaconda3/envs/jupyter_env/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Progress |██████████████████████████████████████████████████| 100.0% Complete
Test complete in 0m 44s
Test Acc: 0.877000
